# Heat transfer in a fin with piecewise variation of some parameters


## Setup


In [ ]:
# # This check is flaky in notebooks
# pyright: reportUnnecessaryTypeIgnoreComment=none

from pathlib import Path
import warnings

import dill  # noqa: S403
from matplotlib import pyplot as plt
import numpy as np
from sympy import (
    Eq,
    FiniteSet,
    Function,
    Piecewise,
    Subs,
    dsolve,
    lambdify,
    pi,
    symbols,
)
from uncertainties import umath

from boilerdata.models.project import Project
from boilerdata.stages.notebooks.modelfun import disp, disp_free

proj = Project.get_project()

## Assign dynamic symbols

We want to use these symbols as individual variable assignments, but they must be in a particular order when we eventually wrap them up in a function for curve fitting. Since `scipy.curve_fit` can't pass keyword arguments, we need to make sure all of our free model parameters come first, then our fixed model parameters. The fixed ones will be handled by `functools.partial()`.


In [ ]:
params = symbols(["x"] + proj.params.free_params + proj.params.fixed_params)

In [ ]:
(
    x,
    T_s,
    q_s,
    h_a,
    k,
    h_w,
) = params

In [ ]:
previously_run_cell_input = _i  # type: ignore  # This is an ugly hack
python_assignment_order = (
    previously_run_cell_input.removeprefix("(\n    ")
    .removesuffix(",\n) = params")
    .split(",\n    ")
)
symbols_order = [p.name for p in params]
print(f"assn={python_assignment_order}", f"syms={symbols_order}", sep="\n")
assert (
    python_assignment_order == symbols_order
), "Variables are NOT in the same order as the symbols to which they are assigned."
print("Variables are in the same order as the symbols to which they are assigned.")

In [ ]:
inputs = symbols(list(proj.params.model_inputs.keys()))

In [ ]:
(
    r,
    T_infa,
    T_infw,
    x_s,
    x_wa,
) = inputs

In [ ]:
previously_run_cell_input = _i  # type: ignore  # This is an ugly hack
python_assignment_order = (
    previously_run_cell_input.removeprefix("(\n    ")
    .removesuffix(",\n) = inputs")
    .split(",\n    ")
)
symbols_order = [p.name for p in inputs]
print(f"assn={python_assignment_order}", f"syms={symbols_order}", sep="\n")
assert (
    python_assignment_order == symbols_order
), "Variables are NOT in the same order as the symbols to which they are assigned."
print("Variables are in the same order as the symbols to which they are assigned.")

## Assign remaining symbols

This symbols fluctuate less, so we can just manually ensure they are in the proper order.

In [ ]:
intermediate_vars = symbols(
    """
    h,
    q_0,
    q_wa,
    T_0,
    T_inf,
    T_wa,
    x_0,
    """
)
(
    h,  # (W/m^2-K) Convection heat transfer coefficient
    q_0,  # (W/m^2) q at x_0, the LHS of a general domain
    q_wa,  # (W/m^2) q at the domain interface
    T_0,  # (C) T at x_0, the LHS of a general domain
    T_inf,  # (C) Ambient temperature
    T_wa,  # (C) T at the domain interface
    x_0,  # (m) x at the LHS of a general domain
) = intermediate_vars

functions = symbols(
    """
    T*,
    T_a,
    T_w,
    T,
    """,
    cls=Function,  # type: ignore  # sympy
)
(
    T_int,  # (T*, C) The general solution to the ODE
    T_a,  # (C) Solution in air
    T_w,  # (C) Solution in water
    T,  # (C) The piecewise combination of the two above solutions
) = functions

for key, val in {
    "Model parameters": params,
    "Model inputs": inputs,
    "Intermediate variables": intermediate_vars,
}.items():
    disp(key, FiniteSet(*val))

disp("Functions", FiniteSet(*(fun(x) for fun in functions)))

## General ODE and its solution


In [ ]:
P = 2 * pi * r
A_c = pi * r**2

ode = Eq(
    T(x).diff(x, 2) - h * P / k / A_c * (T(x) - T_inf),
    0,
)
ics = {
    T(x_0): T_0,
    Subs(T(x).diff(x), x, x_0): q_0 / k,
}
disp("ODE", ode)
disp("Initial conditions", *(Eq(lhs, rhs) for lhs, rhs in ics.items()))

In [ ]:
T_int_expr = dsolve(
    ode,
    T(x),
    ics=ics,
).rhs  # type: ignore  # sympy
disp_free("General solution to the ODE", Eq(T_int(x), T_int_expr))

In [ ]:
# Don't subs/simplify the lhs then try equating to zero. Doesn't work. "Truth value of
# relational" issue. Here we subs/simplify the whole ODE equation.

assert ode.subs(
    T(x), T_int_expr
).simplify(), "The solution to the ODE is not verified by substitution."
print("The solution to the ODE is verified by substitution.")

## Solution in the water domain


In [ ]:
T_w_expr = T_int_expr.subs(
    {
        h: h_w,
        q_0: q_s,
        T_0: T_s,
        T_inf: T_infw,
        x_0: x_s,
    }
)

disp_free("Solution in the water domain", Eq(T_w(x), T_w_expr))

## Values at the domain boundary


In [ ]:
T_wa_expr_w = T_w_expr.subs(x, x_wa)
q_wa_expr_w = (
    T_w_expr.diff(x).subs(  # type: ignore  # sympy
        x,
        x_wa,
    )
    * k
)

disp_free("Temperature at the domain transition", Eq(T_wa, T_wa_expr_w))
disp_free("Heat flux at the domain transition", Eq(q_wa, q_wa_expr_w))

## Solution in the air domain


In [ ]:
T_a_int_expr = T_int_expr.subs(
    {
        h: h_a,
        q_0: q_wa,
        T_0: T_wa,
        T_inf: T_infa,
        x_0: x_wa,
    }
)
T_a_expr = T_a_int_expr.subs(
    {
        q_wa: q_wa_expr_w,
        T_wa: T_wa_expr_w,
    }
)

T_wa_expr_a = T_a_expr.subs(x, x_wa)
q_wa_expr_a = (
    T_a_expr.diff(x).subs(  # type: ignore  # sympy
        x,
        x_wa,
    )
    * k
)

disp_free("Solution in the air domain", Eq(T_a(x), T_a_int_expr))
disp_free("Solution in the air domain, with substitutions", Eq(T_a(x), T_a_expr))

## Check the solution


In [ ]:
assert Eq(
    T_wa_expr_w, T_wa_expr_a
).simplify(), "Temperature discontinuous at domain transition."
assert Eq(
    q_wa_expr_w, q_wa_expr_a
).simplify(), "Temperature gradient discontinuous at domain transition."
print("Temperature and temperature gradient are continuous at the domain transition.")


## Piecewise temperature distribution


In [ ]:
T_expr = Piecewise(
    (T_w_expr, x < x_wa),
    (T_a_expr, True),
)

disp_free("Temperature distribution in the rod", Eq(T(x), T_expr))

## Make the model function compatible with SciPy curve fitting


In [ ]:
class Model:
    cm2_p_m2 = 100**2  # ((cm/m)^2) Conversion factor

    def __init__(self):
        """Model of the temperature distribution in a rod.

        Consists of the lambdified model function and a method for generating a wrapped
        model function that has more flexible input requirements.
        """

        expr = T_expr.evalf(  # type: ignore  # sympy
            subs=proj.params.model_inputs  # type: ignore  # sympy
            | {q_s: q_s * self.cm2_p_m2}  # (W/m^2) = (W/cm^2 * cm^2/m^2)
        )
        disp_free("Temperature distribution after float evaluation", Eq(T(x), expr))

        overrides = {
            ufun.name: np.vectorize(ufun)
            for ufun in (
                umath.exp,  # type: ignore  # uncertainties
                umath.sqrt,  # type: ignore  # uncertainties
            )
        }

        self.basic = lambdify(args=params, expr=expr, modules=np)
        self.for_ufloat = lambdify(args=params, expr=expr, modules=[overrides, np])

In [ ]:
model_for_pickling = Model()
model = model_for_pickling.basic
fig, ax = plt.subplots(layout="constrained")
x_smooth = np.linspace(0, 0.10)
model_kwargs = dict(
    x=x_smooth,
    T_s=105,  # (C)
    q_s=20,  # (W/cm^2)
    h_a=100,  # (W/m^2-K)
    h_w=np.finfo(float).eps,  # (W/m^2-K)
    k=400,  # (W/m-K)
)

model_evaluated_at_x_smooth = model(**model_kwargs)

_ = ax.plot(
    x_smooth,
    model_evaluated_at_x_smooth,
)

## Serialize the model to a file


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", dill.PicklingWarning)
    pickled_model = dill.dumps(model_for_pickling)

Path(proj.dirs.file_model).write_bytes(pickled_model)

with warnings.catch_warnings():
    warnings.simplefilter("ignore", dill.UnpicklingWarning)
    unpickled_model = dill.loads(pickled_model)  # noqa: S301  # Known unpickling.

assert np.allclose(
    model_evaluated_at_x_smooth, unpickled_model.basic(**model_kwargs)
), "The unpickled basic model differs from the original model."

assert np.allclose(
    model_evaluated_at_x_smooth, unpickled_model.for_ufloat(**model_kwargs)
), "The unpickled model for ufloats differs from the original model."

print("The unpickled models are the same as the original model.")